<a href="https://colab.research.google.com/github/aktejo/bcidroneproject/blob/main/EEGNettrue%3Freal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Motor Imagery Task Classification


#Extracting Data

##Downloading BCI Competition IV 2a Dataset

In [ ]:
!wget -r -N -c -np https://physionet.org/files/eegmmidb/1.0.0/

Streaming output truncated to the last 5000 lines.
Length: 2064384 (2.0M) [application/octet-stream]
Saving to: ‘physionet.org/files/eegmmidb/1.0.0/S092/S092R03.edf’

physionet.org/files 100%[===================>]   1.97M  80.2KB/s    in 25s     

2025-05-29 11:41:29 (80.1 KB/s) - ‘physionet.org/files/eegmmidb/1.0.0/S092/S092R03.edf’ saved [2064384/2064384]

--2025-05-29 11:41:29--  https://physionet.org/files/eegmmidb/1.0.0/S092/S092R03.edf.event
Reusing existing connection to physionet.org:443.
HTTP request sent, awaiting response... 200 OK
Length: 566 [text/plain]
Saving to: ‘physionet.org/files/eegmmidb/1.0.0/S092/S092R03.edf.event’

physionet.org/files 100%[===================>]     566  --.-KB/s    in 0s      

2025-05-29 11:41:29 (198 MB/s) - ‘physionet.org/files/eegmmidb/1.0.0/S092/S092R03.edf.event’ saved [566/566]

--2025-05-29 11:41:29--  https://physionet.org/files/eegmmidb/1.0.0/S092/S092R04.edf
Reusing existing connection to physionet.org:443.
HTTP request sent, awaiting 

In [ ]:
!mkdir -p /content/physionet.org/files/cleaned_data/

#Installing Packages

In [ ]:
%%capture
!pip install mne

In [ ]:
%%capture
!pip install torch-summary

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!cp -r /content/physionet.org /content/drive/MyDrive/backup_my_data/

cp: cannot stat '/content/physionet.org': No such file or directory


In [ ]:
!cp -r /content/drive/MyDrive/backup_my_data/files/eegmmidb/1.0.0 /content/physionet.org/

#Libraries Used

In [ ]:
import os
import mne
import math
import copy
import gdown
import random
import scipy.io
import numpy as np
import pandas as pd
import seaborn as sn
import matplotlib.pyplot as plt

# Torch
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchsummary import summary
from torch.autograd import Variable
from torch.utils.data import DataLoader, TensorDataset, random_split

# Scikit-Learn
from sklearn.feature_selection import mutual_info_classif
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.model_selection import train_test_split

# Building Dataset

In [ ]:
!mkdir -p /content/physionet.org/files/cleaned_data/first_session

In [ ]:
!mkdir -p /content/physionet.org/files/cleaned_data/first_session/S001
!mkdir -p /content/physionet.org/files/cleaned_data/first_session/S002
!mkdir -p /content/physionet.org/files/cleaned_data/first_session/S003
!mkdir -p /content/physionet.org/files/cleaned_data/first_session/S004
!mkdir -p /content/physionet.org/files/cleaned_data/first_session/S005
!mkdir -p /content/physionet.org/files/cleaned_data/first_session/S006
!mkdir -p /content/physionet.org/files/cleaned_data/first_session/S007
!mkdir -p /content/physionet.org/files/cleaned_data/first_session/S008
!mkdir -p /content/physionet.org/files/cleaned_data/first_session/S009
!mkdir -p /content/physionet.org/files/cleaned_data/first_session/S010
!mkdir -p /content/physionet.org/files/cleaned_data/first_session/S011
!mkdir -p /content/physionet.org/files/cleaned_data/first_session/S012
!mkdir -p /content/physionet.org/files/cleaned_data/first_session/S013
!mkdir -p /content/physionet.org/files/cleaned_data/first_session/S014
!mkdir -p /content/physionet.org/files/cleaned_data/first_session/S015
!mkdir -p /content/physionet.org/files/cleaned_data/first_session/S016
!mkdir -p /content/physionet.org/files/cleaned_data/first_session/S017
!mkdir -p /content/physionet.org/files/cleaned_data/first_session/S018
!mkdir -p /content/physionet.org/files/cleaned_data/first_session/S019
!mkdir -p /content/physionet.org/files/cleaned_data/first_session/S020
!mkdir -p /content/physionet.org/files/cleaned_data/first_session/S021
!mkdir -p /content/physionet.org/files/cleaned_data/first_session/S022
!mkdir -p /content/physionet.org/files/cleaned_data/first_session/S023
!mkdir -p /content/physionet.org/files/cleaned_data/first_session/S024
!mkdir -p /content/physionet.org/files/cleaned_data/first_session/S025
!mkdir -p /content/physionet.org/files/cleaned_data/first_session/S026
!mkdir -p /content/physionet.org/files/cleaned_data/first_session/S027
!mkdir -p /content/physionet.org/files/cleaned_data/first_session/S028
!mkdir -p /content/physionet.org/files/cleaned_data/first_session/S029
!mkdir -p /content/physionet.org/files/cleaned_data/first_session/S030
!mkdir -p /content/physionet.org/files/cleaned_data/first_session/S031
!mkdir -p /content/physionet.org/files/cleaned_data/first_session/S032
!mkdir -p /content/physionet.org/files/cleaned_data/first_session/S033
!mkdir -p /content/physionet.org/files/cleaned_data/first_session/S034
!mkdir -p /content/physionet.org/files/cleaned_data/first_session/S035
!mkdir -p /content/physionet.org/files/cleaned_data/first_session/S036
!mkdir -p /content/physionet.org/files/cleaned_data/first_session/S037
!mkdir -p /content/physionet.org/files/cleaned_data/first_session/S038
!mkdir -p /content/physionet.org/files/cleaned_data/first_session/S039
!mkdir -p /content/physionet.org/files/cleaned_data/first_session/S040
!mkdir -p /content/physionet.org/files/cleaned_data/first_session/S041
!mkdir -p /content/physionet.org/files/cleaned_data/first_session/S042
!mkdir -p /content/physionet.org/files/cleaned_data/first_session/S043
!mkdir -p /content/physionet.org/files/cleaned_data/first_session/S044
!mkdir -p /content/physionet.org/files/cleaned_data/first_session/S045
!mkdir -p /content/physionet.org/files/cleaned_data/first_session/S046
!mkdir -p /content/physionet.org/files/cleaned_data/first_session/S047
!mkdir -p /content/physionet.org/files/cleaned_data/first_session/S048
!mkdir -p /content/physionet.org/files/cleaned_data/first_session/S049
!mkdir -p /content/physionet.org/files/cleaned_data/first_session/S050
!mkdir -p /content/physionet.org/files/cleaned_data/first_session/S051
!mkdir -p /content/physionet.org/files/cleaned_data/first_session/S052
!mkdir -p /content/physionet.org/files/cleaned_data/first_session/S053
!mkdir -p /content/physionet.org/files/cleaned_data/first_session/S054
!mkdir -p /content/physionet.org/files/cleaned_data/first_session/S055
!mkdir -p /content/physionet.org/files/cleaned_data/first_session/S056
!mkdir -p /content/physionet.org/files/cleaned_data/first_session/S057
!mkdir -p /content/physionet.org/files/cleaned_data/first_session/S058
!mkdir -p /content/physionet.org/files/cleaned_data/first_session/S059
!mkdir -p /content/physionet.org/files/cleaned_data/first_session/S060
!mkdir -p /content/physionet.org/files/cleaned_data/first_session/S061
!mkdir -p /content/physionet.org/files/cleaned_data/first_session/S062
!mkdir -p /content/physionet.org/files/cleaned_data/first_session/S063
!mkdir -p /content/physionet.org/files/cleaned_data/first_session/S064
!mkdir -p /content/physionet.org/files/cleaned_data/first_session/S065
!mkdir -p /content/physionet.org/files/cleaned_data/first_session/S066
!mkdir -p /content/physionet.org/files/cleaned_data/first_session/S067
!mkdir -p /content/physionet.org/files/cleaned_data/first_session/S068
!mkdir -p /content/physionet.org/files/cleaned_data/first_session/S069
!mkdir -p /content/physionet.org/files/cleaned_data/first_session/S070
!mkdir -p /content/physionet.org/files/cleaned_data/first_session/S071
!mkdir -p /content/physionet.org/files/cleaned_data/first_session/S072
!mkdir -p /content/physionet.org/files/cleaned_data/first_session/S073
!mkdir -p /content/physionet.org/files/cleaned_data/first_session/S074
!mkdir -p /content/physionet.org/files/cleaned_data/first_session/S075
!mkdir -p /content/physionet.org/files/cleaned_data/first_session/S076
!mkdir -p /content/physionet.org/files/cleaned_data/first_session/S077
!mkdir -p /content/physionet.org/files/cleaned_data/first_session/S078
!mkdir -p /content/physionet.org/files/cleaned_data/first_session/S079
!mkdir -p /content/physionet.org/files/cleaned_data/first_session/S080
!mkdir -p /content/physionet.org/files/cleaned_data/first_session/S081
!mkdir -p /content/physionet.org/files/cleaned_data/first_session/S082
!mkdir -p /content/physionet.org/files/cleaned_data/first_session/S083
!mkdir -p /content/physionet.org/files/cleaned_data/first_session/S084
!mkdir -p /content/physionet.org/files/cleaned_data/first_session/S085
!mkdir -p /content/physionet.org/files/cleaned_data/first_session/S086
!mkdir -p /content/physionet.org/files/cleaned_data/first_session/S087
!mkdir -p /content/physionet.org/files/cleaned_data/first_session/S088
!mkdir -p /content/physionet.org/files/cleaned_data/first_session/S089
!mkdir -p /content/physionet.org/files/cleaned_data/first_session/S090
!mkdir -p /content/physionet.org/files/cleaned_data/first_session/S091
!mkdir -p /content/physionet.org/files/cleaned_data/first_session/S092
!mkdir -p /content/physionet.org/files/cleaned_data/first_session/S093
!mkdir -p /content/physionet.org/files/cleaned_data/first_session/S094
!mkdir -p /content/physionet.org/files/cleaned_data/first_session/S095
!mkdir -p /content/physionet.org/files/cleaned_data/first_session/S096
!mkdir -p /content/physionet.org/files/cleaned_data/first_session/S097
!mkdir -p /content/physionet.org/files/cleaned_data/first_session/S098
!mkdir -p /content/physionet.org/files/cleaned_data/first_session/S099
!mkdir -p /content/physionet.org/files/cleaned_data/first_session/S101
!mkdir -p /content/physionet.org/files/cleaned_data/first_session/S102
!mkdir -p /content/physionet.org/files/cleaned_data/first_session/S103
!mkdir -p /content/physionet.org/files/cleaned_data/first_session/S104
!mkdir -p /content/physionet.org/files/cleaned_data/first_session/S105
!mkdir -p /content/physionet.org/files/cleaned_data/first_session/S106
!mkdir -p /content/physionet.org/files/cleaned_data/first_session/S107
!mkdir -p /content/physionet.org/files/cleaned_data/first_session/S108
!mkdir -p /content/physionet.org/files/cleaned_data/first_session/S109

##First Session

In [ ]:
import os
import re
import numpy as np
import mne

raw_data_folders = []
cleaned_data_folders = []

for i in range(1, 110):
    if i < 10:
        s = f"S00{i}"
    elif i < 100:
        s = f"S0{i}"
    else:
        s = f"S{i}"

    raw_data_folders.append(f"/content/physionet.org/{s}")
    cleaned_data_folders.append(f"/content/physionet.org/files/cleaned_data/first_session/{s}")

for i in range(109):  # 0 to 108
    files = os.listdir(raw_data_folders[i])
    filtered_files = [file for file in files if file.endswith('.edf')]
    raw_list = []

    for file in filtered_files:
        file_path = os.path.join(raw_data_folders[i], file)

        # Extract run number from file name
        match = re.search(r'R(\d+)\.edf', file)
        run_number = int(match.group(1)) if match else None

        print(f"Loading file: {file} | Run: {run_number}")

        if run_number in [1, 2, 4, 6, 8, 10, 12, 14]:

            raw = mne.io.read_raw_edf(file_path, preload=True)

            # Drop channels (replace with your selected list)
            #raw.drop_channels(['EOG-left', 'EOG-central', 'EOG-right'])  # Example
            # raw.drop_channels([...])  # Add or modify as needed
            raw.drop_channels(['Fc5.', 'Fc1.', 'Fc2.', 'Fc6.', 'C5..', 'C1..', 'C2..', 'C6..', 'Cp5.', 'Cp1.', 'Cp2.', 'Cp6.', 'Fp1.', 'Fpz.', 'Fp2.', 'Af7.', 'Af3.', 'Afz.', 'Af4.', 'Af8.', 'F7..', 'F5..', 'F3..', 'F1..', 'Fz..', 'F2..', 'F4..', 'F6..', 'F8..', 'Ft7.', 'Ft8.', 'T7..', 'T8..', 'T9..', 'T10.', 'Tp7.', 'Tp8.', 'P7..', 'P5..', 'P3..', 'P1..', 'Pz..', 'P2..', 'P4..', 'P6..', 'P8..', 'Po7.', 'Po3.', 'Poz.', 'Po4.', 'Po8.', 'O1..', 'Oz..', 'O2..', 'Iz..'])
            print(raw.ch_names)

            raw.filter(l_freq=4, h_freq=40, method='iir')
            raw.notch_filter(freqs=50)

            # Get events and annotations
            events, _ = mne.events_from_annotations(raw)

            # 1) On the very first raw, grab the target sfreq & channel_order
            if len(raw_list) == 0:
                target_sfreq    = raw.info['sfreq']
                channel_order   = raw.ch_names.copy()
            else:
                # 2) If this raw has a different sfreq, resample it
                if raw.info['sfreq'] != target_sfreq:
                    raw.resample(target_sfreq, npad="auto")

                # 3) Pick only the channels in your original order
                #    (this also drops any extras)
                common = [ch for ch in channel_order if ch in raw.ch_names]
                raw.pick_channels(common)
                # 4) Finally, enforce the original order
                raw.reorder_channels(common)

            # 1) Decide your description-to-description map based on run_number
            if run_number in (1,2,3,5,7,9,11,13,4,8,12):
                # Sessions where t1/t2 mean left/right fist
                desc_map = {'T1': 'T1',      # leave T1 & T2 alone
                            'T2': 'T2'}
            elif run_number in (6,10,14):
                # Sessions where t1/t2 mean both fists/feet
                desc_map = {'T1': 'T3',      # rename the old T1→T3
                            'T2': 'T4'}      # rename the old T2→T4
            else:
                desc_map = {}

            # 2) Apply to raw.annotations directly
            anns = raw.annotations
            new_desc = [ desc_map.get(desc, desc) for desc in anns.description ]
            raw.set_annotations(mne.Annotations(anns.onset, anns.duration, new_desc, orig_time=anns.orig_time))

            # 3) Now re-extract events (they’ll pick up T3/T4)
            events, event_id = mne.events_from_annotations(raw)

            # Save processed file
            os.makedirs(cleaned_data_folders[i], exist_ok=True)
            new_file_path = os.path.join(cleaned_data_folders[i], file[:-4] + '.fif')
            raw.save(new_file_path, overwrite=True)
            raw_list.append(raw)

    if raw_list:
        final_raw = mne.concatenate_raws(raw_list)
        final_path = os.path.join(cleaned_data_folders[i], 'First_Session_Subjects.fif')
        final_raw.save(final_path, overwrite=True)


Loading file: S001R13.edf | Run: 13
Loading file: S001R07.edf | Run: 7
Loading file: S001R04.edf | Run: 4
Extracting EDF parameters from /content/physionet.org/S001/S001R04.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19999  =      0.000 ...   124.994 secs...
['Fc3.', 'Fcz.', 'Fc4.', 'C3..', 'Cz..', 'C4..', 'Cp3.', 'Cpz.', 'Cp4.']
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 4 - 40 Hz

IIR filter parameters
---------------------
Butterworth bandpass zero-phase (two-pass forward and reverse) non-causal filter:
- Filter order 16 (effective, after forward-backward)
- Cutoffs at 4.00, 40.00 Hz: -6.02, -6.02 dB

Filtering raw data in 1 contiguous segment
Setting up band-stop filter from 49 - 51 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB 

[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:    0.0s finished
<ipython-input-20-841846fed351>:89: RuntimeWarning: This filename (/content/physionet.org/files/cleaned_data/first_session/S001/S001R04.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw.save(new_file_path, overwrite=True)
[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:    0.0s finished
<ipython-input-20-841846fed351>:89: RuntimeWarning: This filename (/content/physionet.org/files/cleaned_data/first_session/S001/S001R14.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw.save(new_file_path, overwrite=True)
[Parallel(n_jobs=1)]: Done   9 out of   

- Cutoffs at 4.00, 40.00 Hz: -6.02, -6.02 dB

Filtering raw data in 1 contiguous segment
Setting up band-stop filter from 49 - 51 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 49.38
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 49.12 Hz)
- Upper passband edge: 50.62 Hz
- Upper transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 50.88 Hz)
- Filter length: 1057 samples (6.606 s)

Used Annotations descriptions: [np.str_('T0')]
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
Used Annotations descriptions: [np.str_('T0')]
Overwriting existing file.
Writing /content/physionet.org/files/cleaned_data/first_session/S001/S001R02.fif
Closing /content/physionet.org/files/cleaned_data/first_session/S001/S001R02.fif
[done]
Loading file: S001R0

[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:    0.0s finished
<ipython-input-20-841846fed351>:89: RuntimeWarning: This filename (/content/physionet.org/files/cleaned_data/first_session/S001/S001R02.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw.save(new_file_path, overwrite=True)
[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:    0.0s finished
<ipython-input-20-841846fed351>:89: RuntimeWarning: This filename (/content/physionet.org/files/cleaned_data/first_session/S001/S001R08.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw.save(new_file_path, overwrite=True)
[Parallel(n_jobs=1)]: Done   9 out of   

[done]
Loading file: S001R03.edf | Run: 3
Loading file: S001R12.edf | Run: 12
Extracting EDF parameters from /content/physionet.org/S001/S001R12.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19999  =      0.000 ...   124.994 secs...
['Fc3.', 'Fcz.', 'Fc4.', 'C3..', 'Cz..', 'C4..', 'Cp3.', 'Cpz.', 'Cp4.']
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 4 - 40 Hz

IIR filter parameters
---------------------
Butterworth bandpass zero-phase (two-pass forward and reverse) non-causal filter:
- Filter order 16 (effective, after forward-backward)
- Cutoffs at 4.00, 40.00 Hz: -6.02, -6.02 dB

Filtering raw data in 1 contiguous segment
Setting up band-stop filter from 49 - 51 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower

...

[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:    0.0s finished
<ipython-input-20-841846fed351>:89: RuntimeWarning: This filename (/content/physionet.org/files/cleaned_data/first_session/S109/S109R06.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw.save(new_file_path, overwrite=True)
[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:    0.0s finished
<ipython-input-20-841846fed351>:89: RuntimeWarning: This filename (/content/physionet.org/files/cleaned_data/first_session/S109/S109R04.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw.save(new_file_path, overwrite=True)
[Parallel(n_jobs=1)]: Done   9 out of   

[done]
Loading file: S109R12.edf | Run: 12
Extracting EDF parameters from /content/physionet.org/S109/S109R12.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19679  =      0.000 ...   122.994 secs...
['Fc3.', 'Fcz.', 'Fc4.', 'C3..', 'Cz..', 'C4..', 'Cp3.', 'Cpz.', 'Cp4.']
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 4 - 40 Hz

IIR filter parameters
---------------------
Butterworth bandpass zero-phase (two-pass forward and reverse) non-causal filter:
- Filter order 16 (effective, after forward-backward)
- Cutoffs at 4.00, 40.00 Hz: -6.02, -6.02 dB

Filtering raw data in 1 contiguous segment
Setting up band-stop filter from 49 - 51 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 49.38
- Lower trans

[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:    0.0s finished
<ipython-input-20-841846fed351>:89: RuntimeWarning: This filename (/content/physionet.org/files/cleaned_data/first_session/S109/S109R12.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw.save(new_file_path, overwrite=True)
[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:    0.0s finished
<ipython-input-20-841846fed351>:89: RuntimeWarning: This filename (/content/physionet.org/files/cleaned_data/first_session/S109/S109R10.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw.save(new_file_path, overwrite=True)
[Parallel(n_jobs=1)]: Done   9 out of   

[done]
Loading file: S109R13.edf | Run: 13
Loading file: S109R09.edf | Run: 9
Loading file: S109R11.edf | Run: 11
Loading file: S109R05.edf | Run: 5
Loading file: S109R07.edf | Run: 7
Loading file: S109R01.edf | Run: 1
Extracting EDF parameters from /content/physionet.org/S109/S109R01.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 9599  =      0.000 ...    59.994 secs...
['Fc3.', 'Fcz.', 'Fc4.', 'C3..', 'Cz..', 'C4..', 'Cp3.', 'Cpz.', 'Cp4.']
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 4 - 40 Hz

IIR filter parameters
---------------------
Butterworth bandpass zero-phase (two-pass forward and reverse) non-causal filter:
- Filter order 16 (effective, after forward-backward)
- Cutoffs at 4.00, 40.00 Hz: -6.02, -6.02 dB

Filtering raw data in 1 contiguous segment
Setting up band-stop filter from 49 - 51 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandst

[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:    0.0s finished
<ipython-input-20-841846fed351>:89: RuntimeWarning: This filename (/content/physionet.org/files/cleaned_data/first_session/S109/S109R01.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw.save(new_file_path, overwrite=True)
<ipython-input-20-841846fed351>:95: RuntimeWarning: This filename (/content/physionet.org/files/cleaned_data/first_session/S109/First_Session_Subjects.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  final_raw.save(final_path, overwrite=True)


In [ ]:
# import os
# import re
# import numpy as np
# import mne

# raw_data_folders = []
# cleaned_data_folders = []

# for i in range(1, 110):
#     if i < 10:
#         s = f"S00{i}"
#     elif i < 100:
#         s = f"S0{i}"
#     else:
#         s = f"S{i}"

#     raw_data_folders.append(f"/content/physionet.org/{s}")
#     cleaned_data_folders.append(f"/content/physionet.org/files/cleaned_data/first_session/{s}")

# for i in range(109):  # 0 to 108
#     files = os.listdir(raw_data_folders[i])
#     filtered_files = [file for file in files if file.endswith('.edf')]
#     raw_list = []

#     for file in filtered_files:
#         file_path = os.path.join(raw_data_folders[i], file)

#         # Extract run number from file name
#         match = re.search(r'R(\d+)\.edf', file)
#         run_number = int(match.group(1)) if match else None

#         print(f"Loading file: {file} | Run: {run_number}")

#         if run_number in [1, 2, 4, 6, 8, 10, 12, 14]:

#             raw = mne.io.read_raw_edf(file_path, preload=True)

#             # Drop channels (replace with your selected list)
#             #raw.drop_channels(['EOG-left', 'EOG-central', 'EOG-right'])  # Example
#             # raw.drop_channels([...])  # Add or modify as needed
#             raw.drop_channels(['Fc5.', 'Fc1.', 'Fc2.', 'Fc6.', 'C5..', 'C1..', 'C2..', 'C6..', 'Cp5.', 'Cp1.', 'Cp2.', 'Cp6.', 'Fp1.', 'Fpz.', 'Fp2.', 'Af7.', 'Af3.', 'Afz.', 'Af4.', 'Af8.', 'F7..', 'F5..', 'F3..', 'F1..', 'Fz..', 'F2..', 'F4..', 'F6..', 'F8..', 'Ft7.', 'Ft8.', 'T7..', 'T8..', 'T9..', 'T10.', 'Tp7.', 'Tp8.', 'P7..', 'P5..', 'P3..', 'P1..', 'Pz..', 'P2..', 'P4..', 'P6..', 'P8..', 'Po7.', 'Po3.', 'Poz.', 'Po4.', 'Po8.', 'O1..', 'Oz..', 'O2..', 'Iz..'])
#             print(raw.ch_names)

#             raw.filter(l_freq=4, h_freq=40, method='iir')
#             raw.notch_filter(freqs=50)

#             # Get events and annotations
#             events, _ = mne.events_from_annotations(raw)

#             # Mapping logic
#             task_map = {
#                 (1, 2, 3, 5, 7, 9, 11, 13, 4, 8, 12): {1: 1, 2: 2},  # left/right fist
#                 (6, 10, 14): {1: 3, 2: 4}  # both fists/feet
#             }

#             # # Event and annotation mapping
#             # events, event_dict = mne.events_from_annotations(raw)
#             # label_map = task_map[run_number]
#             # selected_event_id = {k: event_dict[k] for k in label_map if k in event_dict}
#             # if not selected_event_id:
#             #     continue

#             # code_to_annot = {v: k for k, v in event_dict.items()}
#             # mapped_labels = [label_map[code_to_annot[code]] for code in labels]

#             label_map = None
#             for runs, mapping in task_map.items():
#                 if run_number in runs:
#                     label_map = mapping
#                     break

#             if label_map is None:
#                 raise ValueError(f"Run number {run_number} is not in the defined task mapping.")

#             T0_CODE = 0
#             remapped_events = []
#             for ev in events:
#                 code = ev[2]
#                 if code == T0_CODE:
#                     continue
#                 new_label = label_map.get(code)
#                 if new_label:
#                     remapped_events.append([ev[0], ev[1], new_label])

#             remapped_events = np.array(remapped_events)

#             # Save processed file
#             os.makedirs(cleaned_data_folders[i], exist_ok=True)
#             new_file_path = os.path.join(cleaned_data_folders[i], file[:-4] + '.fif')
#             raw.save(new_file_path, overwrite=True)
#             raw_list.append(raw)

#     if raw_list:
#         final_raw = mne.concatenate_raws(raw_list)
#         final_path = os.path.join(cleaned_data_folders[i], 'First_Session_Subjects.fif')
#         final_raw.save(final_path, overwrite=True)


Loading file: S001R13.edf | Run: 13
Loading file: S001R07.edf | Run: 7
Loading file: S001R04.edf | Run: 4
Extracting EDF parameters from /content/physionet.org/S001/S001R04.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19999  =      0.000 ...   124.994 secs...
['Fc3.', 'Fcz.', 'Fc4.', 'C3..', 'Cz..', 'C4..', 'Cp3.', 'Cpz.', 'Cp4.']
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 4 - 40 Hz

IIR filter parameters
---------------------
Butterworth bandpass zero-phase (two-pass forward and reverse) non-causal filter:
- Filter order 16 (effective, after forward-backward)
- Cutoffs at 4.00, 40.00 Hz: -6.02, -6.02 dB

Filtering raw data in 1 contiguous segment
Setting up band-stop filter from 49 - 51 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB 

[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:    0.0s finished


KeyError: 4

In [ ]:
# import os
# import re
# import numpy as np
# import mne

# raw_data_folders = []
# cleaned_data_folders = []

# for i in range(1, 110):
#     if i < 10:
#         s = f"S00{i}"
#     elif i < 100:
#         s = f"S0{i}"
#     else:
#         s = f"S{i}"

#     raw_data_folders.append(f"/content/physione\t.org/files/eegmmidb/1.0.0/{s}")
#     cleaned_data_folders.append(f"/content/physionet.org/files/cleaned_data/first_session/{s}")

# for i in range(109):  # 0 to 108
#     files = os.listdir(raw_data_folders[i])
#     filtered_files = [file for file in files if file.endswith('.edf')]
#     raw_list = []

#     for file in filtered_files:
#         file_path = os.path.join(raw_data_folders[i], file)

#         # Extract run number from file name
#         match = re.search(r'R(\d+)\.edf', file)
#         run_number = int(match.group(1)) if match else None

#         print(f"Loading file: {file} | Run: {run_number}")

#         if run_number in [1, 2, 4, 6, 8, 10, 12, 14]:

#             raw = mne.io.read_raw_edf(file_path, preload=True)

#             # Drop channels (replace with your selected list)
#             #raw.drop_channels(['EOG-left', 'EOG-central', 'EOG-right'])  # Example
#             # raw.drop_channels([...])  # Add or modify as needed
#             raw.drop_channels(['Fc5.', 'Fc1.', 'Fc2.', 'Fc6.', 'C5..', 'C1..', 'C2..', 'C6..', 'Cp5.', 'Cp1.', 'Cp2.', 'Cp6.', 'Fp1.', 'Fpz.', 'Fp2.', 'Af7.', 'Af3.', 'Afz.', 'Af4.', 'Af8.', 'F7..', 'F5..', 'F3..', 'F1..', 'Fz..', 'F2..', 'F4..', 'F6..', 'F8..', 'Ft7.', 'Ft8.', 'T7..', 'T8..', 'T9..', 'T10.', 'Tp7.', 'Tp8.', 'P7..', 'P5..', 'P3..', 'P1..', 'Pz..', 'P2..', 'P4..', 'P6..', 'P8..', 'Po7.', 'Po3.', 'Poz.', 'Po4.', 'Po8.', 'O1..', 'Oz..', 'O2..', 'Iz..'])
#             print(raw.ch_names)

#             raw.filter(l_freq=4, h_freq=40, method='iir')
#             raw.notch_filter(freqs=50)

#             # Get events and annotations
#             events, _ = mne.events_from_annotations(raw)

#             # Mapping logic
#             task_map = {
#                 (1, 2, 3, 5, 7, 9, 11, 13, 4, 8, 12): {1: 1, 2: 2},  # left/right fist
#                 (6, 10, 14): {1: 3, 2: 4}  # both fists/feet
#             }

#             # Event and annotation mapping
#             events, event_dict = mne.events_from_annotations(raw)
#             label_map = task_map[run_number]
#             selected_event_id = {k: event_dict[k] for k in label_map if k in event_dict}
#             if not selected_event_id:
#                 continue

#             code_to_annot = {v: k for k, v in event_dict.items()}
#             mapped_labels = [label_map[code_to_annot[code]] for code in labels]

#             label_map = None
#             for runs, mapping in task_map.items():
#                 if run_number in runs:
#                     label_map = mapping
#                     break

#             if label_map is None:
#                 raise ValueError(f"Run number {run_number} is not in the defined task mapping.")

#             T0_CODE = 0
#             remapped_events = []
#             for ev in events:
#                 code = ev[2]
#                 if code == T0_CODE:
#                     continue
#                 new_label = label_map.get(code)
#                 if new_label:
#                     remapped_events.append([ev[0], ev[1], new_label])

#             remapped_events = np.array(remapped_events)

#             # Save processed file
#             os.makedirs(cleaned_data_folders[i], exist_ok=True)
#             new_file_path = os.path.join(cleaned_data_folders[i], file[:-4] + '.fif')
#             raw.save(new_file_path, overwrite=True)
#             raw_list.append(raw)

#     if raw_list:
#         final_raw = mne.concatenate_raws(raw_list)
#         final_path = os.path.join(cleaned_data_folders[i], 'First_Session_Subjects.fif')
#         final_raw.save(final_path, overwrite=True)

**List of the events**  
'1023': 1 Rejected trial  
'1072': 2 Eye movements  
'276': 3 Idling EEG (eyes open)  
'277': 4 Idling EEG (eyes closed)  
'32766': 5 Start of a new run  
'768': 6 Start of a trial  
'769': 7 Cue onset **Left** (class 1) : 0  
'770': 8 Cue onset **Right** (class 2) : 1  
'771': 9 Cue onset **Foot** (class 3) : 2  
'772': 10 Cue onset **Tongue** (class 4): 3

In [ ]:
events = mne.events_from_annotations(final_raw)
events[1]

Used Annotations descriptions: [np.str_('T0'), np.str_('T1'), np.str_('T2'), np.str_('T3'), np.str_('T4')]


{np.str_('T0'): 1,
 np.str_('T1'): 2,
 np.str_('T2'): 3,
 np.str_('T3'): 4,
 np.str_('T4'): 5}

**Time choice:**  
[0.s, 4s] Post Cue on set:  [2s, 6s]

In [ ]:
events

(array([[     0,      0,      1],
        [  9760,      0,      1],
        [ 10416,      0,      5],
        [ 11072,      0,      1],
        [ 11728,      0,      4],
        [ 12384,      0,      1],
        [ 13040,      0,      4],
        [ 13696,      0,      1],
        [ 14352,      0,      5],
        [ 15008,      0,      1],
        [ 15664,      0,      5],
        [ 16320,      0,      1],
        [ 16976,      0,      4],
        [ 17632,      0,      1],
        [ 18288,      0,      4],
        [ 18944,      0,      1],
        [ 19600,      0,      5],
        [ 20256,      0,      1],
        [ 20912,      0,      4],
        [ 21568,      0,      1],
        [ 22224,      0,      5],
        [ 22880,      0,      1],
        [ 23536,      0,      4],
        [ 24192,      0,      1],
        [ 24848,      0,      5],
        [ 25504,      0,      1],
        [ 26160,      0,      5],
        [ 26816,      0,      1],
        [ 27472,      0,      4],
        [ 2812

In [ ]:
epochs = mne.Epochs(final_raw, events[0], event_id=[1, 2, 3, 4, 5], tmin=0, tmax=4, reject=None, baseline=None, preload=True)
first_session_data = epochs.get_data(copy=True)
first_session_labels = epochs.events[:,-1]

Not setting metadata
182 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 182 events and 641 original time points ...
0 bad epochs dropped


In [ ]:
print("First_session_dataset shape:",first_session_data.shape)
print("First_session_labels shape:", first_session_labels.shape)

First_session_dataset shape: (137, 9, 641)
First_session_labels shape: (137,)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


###Structuring Data

In [ ]:
# Choosing Device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Loss Function
criterion = nn.CrossEntropyLoss()

# Normalizing Labels to [0, 1, 2, 3]
y_train = first_session_labels - np.min(first_session_labels)
#y_test = second_session_labels - np.min(second_session_labels)

# Normalizing Input features: z-score(mean=0, std=1)
X_first_session = (first_session_data - np.mean(first_session_data)) / np.std(first_session_data)
#X_second_session = (second_session_data - np.mean(second_session_data)) / np.std(second_session_data)

X = X_first_session
print(X.shape)
y = y_train

# Spliting  Data: 90% for Train and 10% for Test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42, stratify=y)

# Converting to Tensor
X_train = torch.Tensor(X_train).unsqueeze(1).to(device)
X_test = torch.Tensor(X_test).unsqueeze(1).to(device)
y_train = torch.LongTensor(y_train).to(device)
y_test = torch.LongTensor(y_test).to(device)

# Creating Tensor Dataset
train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)

# Printing the sizes
print("Size of X_train:", X_train.size())
print("Size of X_test:", X_test.size())
print("Size of y_train:", y_train.size())
print("Size of y_test:", y_test.size())


(137, 9, 641)
Size of X_train: torch.Size([123, 1, 9, 641])
Size of X_test: torch.Size([14, 1, 9, 641])
Size of y_train: torch.Size([123])
Size of y_test: torch.Size([14])


#Training Class

In [ ]:
class TrainModel():
    def __init__(self,):
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    def train_model(self, model, train_dataset, learning_rate=0.001, batch_size=64, epochs=500):
        model = model.to(self.device)
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.Adam(model.parameters(), lr=learning_rate)
        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
        highest_train_accuracy = 0.0

        for epoch in range(epochs):
            model.train()
            running_loss = 0.0
            correct = 0
            total = 0
            for inputs, labels in train_loader:
                inputs = inputs.to(self.device)
                labels = labels.to(self.device)

                optimizer.zero_grad()
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()

                running_loss += loss.item() * inputs.size(0)
                _, predicted = torch.max(outputs, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

            epoch_loss = running_loss / len(train_loader.dataset)
            epoch_accuracy = correct / total
            if epoch_accuracy > highest_train_accuracy:
                highest_train_accuracy = epoch_accuracy
            print(f"Epoch {epoch+1}/{epochs}, Loss: {epoch_loss:.4f}, Accuracy: {(epoch_accuracy*100):.2f}%")

        average_loss = running_loss / len(train_loader.dataset)
        print("Average Loss:", average_loss)
        print("Highest Train Accuracy:", highest_train_accuracy)

        # Saving model
        torch.save(model.state_dict(), '/content/eegnet_model.pth')
        return model


#Evaluating Model

In [ ]:
class EvalModel():
    def __init__(self, model):
        self.model = model.to(device)
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    def test_model(self, test_dataset):
        self.model.eval()
        correct = 0
        total = 0
        test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

        with torch.no_grad():
            for inputs, labels in test_loader:
                inputs = inputs.to(self.device)
                labels = labels.to(self.device)
                outputs = self.model(inputs)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        accuracy = (correct / total) * 100
        print("/------------------------------/")
        print(f"Test Accuracy: {accuracy:.2f}%")
        print("/------------------------------/")
        return accuracy

    def plot_confusion_matrix(self, test_dataset, classes):
        self.model.eval()
        y_pred = []
        y_true = []
        test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

        with torch.no_grad():
            for inputs, labels in test_loader:
                inputs = inputs.to(self.device)
                labels = labels.to(self.device)
                outputs = self.model(inputs)
                _, predicted = torch.max(outputs.data, 1)
                y_pred.append(predicted.item())
                y_true.append(labels.item())

        cf_matrix = confusion_matrix(y_true, y_pred)
        cf_matrix = cf_matrix.astype('float') / cf_matrix.sum(axis=1)[:, np.newaxis]

        df_cm = pd.DataFrame(cf_matrix, index=classes, columns=classes)

        plt.figure(figsize=(10, 7))
        sn.heatmap(df_cm, annot=True, cmap='Blues', fmt='.2f')
        plt.xlabel('Predicted labels')
        plt.ylabel('True labels')
        plt.title('Confusion Matrix')
        plt.savefig('/content/physionet.org/confusion_matrix_model.png')
        plt.show()


#EEGNet Model

In [ ]:
class EEGNetModel(nn.Module):
    def __init__(self, chans=9, classes=5, time_points=641, temp_kernel=32,
                 f1=16, f2=32, d=2, pk1=8, pk2=16, dropout_rate=0.5,
                 max_norm1=1, max_norm2=0.25):
        super(EEGNetModel, self).__init__()
        self.block1 = nn.Sequential(
            nn.Conv2d(1, f1, (1, temp_kernel), padding='same', bias=False),
            nn.BatchNorm2d(f1),
        )
        self.block2 = nn.Sequential(
            nn.Conv2d(f1, d*f1, (chans,1), groups=f1, bias=False),
            nn.BatchNorm2d(d*f1),
            nn.ELU(),
            nn.AvgPool2d((1, pk1)),
            nn.Dropout(dropout_rate)
        )
        self.block3 = nn.Sequential(
            nn.Conv2d(d*f1, f2, (1,16), groups=f2, bias=False, padding='same'),
            nn.Conv2d(f2, f2, 1, bias=False),
            nn.BatchNorm2d(f2),
            nn.ELU(),
            nn.AvgPool2d((1, pk2)),
            nn.Dropout(dropout_rate)
        )

        # figure out flatten size automatically
        with torch.no_grad():
            x = torch.zeros(1, 1, chans, time_points)
            x = self.block1(x)
            x = self.block2(x)
            x = self.block3(x)
            flatten_dim = x.flatten(1).shape[1]

        self.flatten = nn.Flatten()
        self.fc      = nn.Linear(flatten_dim, classes)

        # apply max‐norm as before…
        self._apply_max_norm(self.block2[0], max_norm1)
        self._apply_max_norm(self.fc,     max_norm2)

    def _apply_max_norm(self, layer, max_norm):
      for name, param in layer.named_parameters():
          if 'weight' in name:
              param.data = torch.renorm(param.data, p=2, dim=0, maxnorm=max_norm)

    def forward(self, x):
        x = self.block1(x)
        x = self.block2(x)
        x = self.block3(x)
        x = self.flatten(x)
        x = self.fc(x)
        return x


In [ ]:
# class EEGNetModel(nn.Module): # EEGNET-8,2
#     def __init__(self, chans=9, classes=4, time_points=641, temp_kernel=32,
#                  f1=16, f2=32, d=2, pk1=8, pk2=16, dropout_rate=0.5, max_norm1=1, max_norm2=0.25):
#         super(EEGNetModel, self).__init__()
#         # Calculating FC input features
#         linear_size = (time_points//(pk1*pk2))*f2

#         # Temporal Filters
#         self.block1 = nn.Sequential(
#             nn.Conv2d(1, f1, (1, temp_kernel), padding='same', bias=False),
#             nn.BatchNorm2d(f1),
#         )
#         # Spatial Filters
#         self.block2 = nn.Sequential(
#             nn.Conv2d(f1, d * f1, (chans, 1), groups=f1, bias=False), # Depthwise Conv
#             nn.BatchNorm2d(d * f1),
#             nn.ELU(),
#             nn.AvgPool2d((1, pk1)),
#             nn.Dropout(dropout_rate)
#         )
#         self.block3 = nn.Sequential(
#             nn.Conv2d(d * f1, f2, (1, 16),  groups=f2, bias=False, padding='same'), # Separable Conv
#             nn.Conv2d(f2, f2, kernel_size=1, bias=False), # Pointwise Conv
#             nn.BatchNorm2d(f2),
#             nn.ELU(),
#             nn.AvgPool2d((1, pk2)),
#             nn.Dropout(dropout_rate)
#         )

#         self.flatten = nn.Flatten()
#         self.fc = nn.Linear(linear_size, classes)

#         # Apply max_norm constraint to the depthwise layer in block2
#         self._apply_max_norm(self.block2[0], max_norm1)

#         # Apply max_norm constraint to the linear layer
#         self._apply_max_norm(self.fc, max_norm2)

#     def _apply_max_norm(self, layer, max_norm):
#         for name, param in layer.named_parameters():
#             if 'weight' in name:
#                 param.data = torch.renorm(param.data, p=2, dim=0, maxnorm=max_norm)

#     def forward(self, x):
#         x = self.block1(x)
#         x = self.block2(x)
#         x = self.block3(x)
#         x = self.flatten(x)
#         x = self.fc(x)
#         return x




##Model Summery

In [ ]:
input_size = (1, 9, 641)
#input_size = (1,22,1001)
eegnet_model = EEGNetModel().to(device)
summary(eegnet_model, input_size)

Layer (type:depth-idx)                   Output Shape              Param #
├─Sequential: 1-1                        [-1, 16, 9, 641]          --
|    └─Conv2d: 2-1                       [-1, 16, 9, 641]          512
|    └─BatchNorm2d: 2-2                  [-1, 16, 9, 641]          32
├─Sequential: 1-2                        [-1, 32, 1, 80]           --
|    └─Conv2d: 2-3                       [-1, 32, 1, 641]          288
|    └─BatchNorm2d: 2-4                  [-1, 32, 1, 641]          64
|    └─ELU: 2-5                          [-1, 32, 1, 641]          --
|    └─AvgPool2d: 2-6                    [-1, 32, 1, 80]           --
|    └─Dropout: 2-7                      [-1, 32, 1, 80]           --
├─Sequential: 1-3                        [-1, 32, 1, 5]            --
|    └─Conv2d: 2-8                       [-1, 32, 1, 80]           512
|    └─Conv2d: 2-9                       [-1, 32, 1, 80]           1,024
|    └─BatchNorm2d: 2-10                 [-1, 32, 1, 80]           64
|    └─EL

Layer (type:depth-idx)                   Output Shape              Param #
├─Sequential: 1-1                        [-1, 16, 9, 641]          --
|    └─Conv2d: 2-1                       [-1, 16, 9, 641]          512
|    └─BatchNorm2d: 2-2                  [-1, 16, 9, 641]          32
├─Sequential: 1-2                        [-1, 32, 1, 80]           --
|    └─Conv2d: 2-3                       [-1, 32, 1, 641]          288
|    └─BatchNorm2d: 2-4                  [-1, 32, 1, 641]          64
|    └─ELU: 2-5                          [-1, 32, 1, 641]          --
|    └─AvgPool2d: 2-6                    [-1, 32, 1, 80]           --
|    └─Dropout: 2-7                      [-1, 32, 1, 80]           --
├─Sequential: 1-3                        [-1, 32, 1, 5]            --
|    └─Conv2d: 2-8                       [-1, 32, 1, 80]           512
|    └─Conv2d: 2-9                       [-1, 32, 1, 80]           1,024
|    └─BatchNorm2d: 2-10                 [-1, 32, 1, 80]           64
|    └─EL

##Training Model

In [ ]:
eegnet_model = EEGNetModel().to(device)

# Training Hyperparameters
EPOCHS = 500
BATCH_SIZE = 64
LEARNING_RATE = 0.001
trainer = TrainModel()
trained_eegnet_model = trainer.train_model(eegnet_model, train_dataset, learning_rate=LEARNING_RATE,
                                   batch_size=BATCH_SIZE, epochs=EPOCHS)
torch.save(trained_eegnet_model.state_dict(), '/content/eegnet_model.pth')


Epoch 1/500, Loss: 1.6236, Accuracy: 10.57%
Epoch 2/500, Loss: 1.5698, Accuracy: 31.71%
Epoch 3/500, Loss: 1.5428, Accuracy: 43.09%
Epoch 4/500, Loss: 1.4984, Accuracy: 56.91%
Epoch 5/500, Loss: 1.4478, Accuracy: 65.85%
Epoch 6/500, Loss: 1.4044, Accuracy: 67.48%
Epoch 7/500, Loss: 1.3674, Accuracy: 66.67%
Epoch 8/500, Loss: 1.3300, Accuracy: 66.67%
Epoch 9/500, Loss: 1.2883, Accuracy: 66.67%
Epoch 10/500, Loss: 1.2553, Accuracy: 66.67%
Epoch 11/500, Loss: 1.2193, Accuracy: 66.67%
Epoch 12/500, Loss: 1.1927, Accuracy: 66.67%
Epoch 13/500, Loss: 1.1720, Accuracy: 66.67%
Epoch 14/500, Loss: 1.1469, Accuracy: 66.67%
Epoch 15/500, Loss: 1.0993, Accuracy: 66.67%
Epoch 16/500, Loss: 1.0976, Accuracy: 66.67%
Epoch 17/500, Loss: 1.0818, Accuracy: 66.67%
Epoch 18/500, Loss: 1.0647, Accuracy: 66.67%
Epoch 19/500, Loss: 1.0318, Accuracy: 66.67%
Epoch 20/500, Loss: 1.0366, Accuracy: 66.67%
Epoch 21/500, Loss: 1.0033, Accuracy: 66.67%
Epoch 22/500, Loss: 0.9920, Accuracy: 66.67%
Epoch 23/500, Loss:

##Evaluating Model

In [ ]:
import numpy as np
unique, counts = np.unique(y_true, return_counts=True)
print("Test-set labels present:", dict(zip(unique, counts)))

NameError: name 'y_true' is not defined

In [ ]:
classes_list = [1, 2, 3, 4, 5]
eval_model = EvalModel(trained_eegnet_model)
test_accuracy = eval_model.test_model(test_dataset)
eval_model.plot_confusion_matrix(test_dataset, classes_list)

/------------------------------/
Test Accuracy: 64.29%
/------------------------------/


ValueError: Shape of passed values is (3, 3), indices imply (5, 5)